# Simple character level tokenizer

In [343]:
with open('shakespeare.txt', 'r') as f:
    text = f.read()
chars = sorted(list(set(text)))
print("\nVocabulary size is ", len(chars), "chars/tokens: ", ''.join(chars))

ctoi, itoc = dict((char, index) for index, char in enumerate(chars)), dict((index, char) for index, char in enumerate(chars))
print("\nVocabulary: ",  ctoi)

encode = lambda sentence: [ctoi[char] for char in sentence] # encode sentence to index
decode = lambda indexes: ''.join([itoc[index] for index in indexes]) # decode index to sentence
print("example: ",encode("some text for training"), decode(encode("some text for training")))


Vocabulary size is  65 chars/tokens:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

Vocabulary:  {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
example:  [57, 53, 51, 43, 1, 58, 43, 62, 58, 1, 44, 53, 56, 1, 58, 56, 39, 47, 52, 47, 52, 45] some text for training


In [344]:
import torch
data = torch.LongTensor(encode(text))
print("\nData shape:", data.shape, "\nData type:", data.dtype)
print("Data example:\n", data[:])


Data shape: torch.Size([1115393]) 
Data type: torch.int64
Data example:
 tensor([18, 47, 56,  ..., 52, 45,  8])


# Separate data to train and validation 90/10
### generate a small batch of data of inputs x and targets y

In [345]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [346]:
# block size
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input is {context} and target is {target}")

input is tensor([18]) and target is 47
input is tensor([18, 47]) and target is 56
input is tensor([18, 47, 56]) and target is 57
input is tensor([18, 47, 56, 57]) and target is 58
input is tensor([18, 47, 56, 57, 58]) and target is 1
input is tensor([18, 47, 56, 57, 58,  1]) and target is 15
input is tensor([18, 47, 56, 57, 58,  1, 15]) and target is 47
input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target is 58


In [347]:
# Batch size dimension
torch.manual_seed(1337) # for reproducibility (same random numbers every time)
batch_size = 4 # number of independent sequences to train on in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"input is {context.tolist()} and target is {target}")


inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----
input is [53] and target is 59
input is [53, 59] and target is 6
input is [53, 59, 6] and target is 1
input is [53, 59, 6, 1] and target is 58
input is [53, 59, 6, 1, 58] and target is 56
input is [53, 59, 6, 1, 58, 56] and target is 47
input is [53, 59, 6, 1, 58, 56, 47] and target is 40
input is [53, 59, 6, 1, 58, 56, 47, 40] and target is 59
input is [49] and target is 43
input is [49, 43] and target is 43
input is [49, 43, 43] and target is 54
input is [49, 43, 43, 54] and target is 1
input is [49, 43, 43, 54, 1] and target is 47
input is [49, 43, 43, 54, 1, 47] and target is 58
input is [49

In [348]:
print(xb) # input to the transformer

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
